# professor_guide (Colab Auto-Setup)

This notebook is Colab-ready and will automatically:
1. Detect Colab
2. (Optional) mount Google Drive for persistence
3. Clone **or** pull the full GitHub repository
4. Install dependencies (supports `requirements.txt`, `setup.py`, and optional Poetry projects)
5. Configure `PYTHONPATH` for both repo-root and `src/` layouts

## Quick start
- Update **`REPO_URL`** and **`REPO_DIR`** in the first code cell.
- (Optional) set **`USE_DRIVE = True`** if you want the repo to persist across Colab sessions.


In [5]:
# =========================
# Colab Auto-Setup (Repo)
# - clone/pull repo
# - cd into repo
# =========================
import os, sys, subprocess, pathlib

def in_colab() -> bool:
    return ("COLAB_GPU" in os.environ) or ("google.colab" in sys.modules)

# -------------------------
# EDIT THESE
# -------------------------
REPO_URL = "https://github.com/tommasocarzaniga/CNM_CycNucMed.git"
REPO_DIR = "CNM"

# If True, clone into Google Drive for persistence between sessions.
USE_DRIVE = False

def run(cmd, cwd=None):
    print(">>", " ".join(cmd))
    subprocess.check_call(cmd, cwd=cwd)

if in_colab():
    print("Running on Colab ✅")

    base = "/content"
    if USE_DRIVE:
        from google.colab import drive
        drive.mount("/content/drive")
        base = "/content/drive/MyDrive"  # adjust if you prefer another folder

    repo_path = os.path.join(base, REPO_DIR)

    # Clone or pull
    if not os.path.exists(repo_path):
        os.makedirs(base, exist_ok=True)
        run(["git", "clone", REPO_URL, REPO_DIR], cwd=base)
    else:
        # In case repo exists but isn't a git repo, fail loudly
        if not os.path.exists(os.path.join(repo_path, ".git")):
            raise RuntimeError(f"{repo_path} exists but is not a git repository (missing .git)")
        run(["git", "pull"], cwd=repo_path)

    # Change into repo folder
    os.chdir(repo_path)
    print("Repo path:", os.getcwd())
else:
    print("Not running on Colab (local/Jupyter) ✅")
    print("Working dir:", os.getcwd())


Running on Colab ✅
>> git clone https://github.com/tommasocarzaniga/CNM_CycNucMed.git REPO
Repo path: /content/REPO


In [6]:
# =========================
# Install dependencies
# - requirements.txt
# - setup.py (editable)
# - pyproject.toml (optional Poetry)
# =========================
import sys, subprocess, pathlib

def run(cmd):
    print(">>", " ".join(cmd))
    subprocess.check_call(cmd)

root = pathlib.Path(".").resolve()

# Always keep pip/setuptools reasonably fresh in Colab
run([sys.executable, "-m", "pip", "install", "-U", "pip", "setuptools", "wheel"])

# 1) requirements.txt
req = root / "requirements.txt"
if req.exists():
    run([sys.executable, "-m", "pip", "install", "-r", str(req)])
else:
    print("No requirements.txt found (skipping)")

# 2) setup.py (editable install)
setup_py = root / "setup.py"
if setup_py.exists():
    run([sys.executable, "-m", "pip", "install", "-e", "."])
else:
    print("No setup.py found (skipping editable install)")

# 3) pyproject.toml (Poetry optional)
pyproject = root / "pyproject.toml"
if pyproject.exists():
    print("Found pyproject.toml.")
    print("If this project uses Poetry, uncomment the lines below:")
    print("  !pip install poetry")
    print("  !poetry install")


>> /usr/bin/python3 -m pip install -U pip setuptools wheel
>> /usr/bin/python3 -m pip install -r /content/REPO/requirements.txt
No setup.py found (skipping editable install)
Found pyproject.toml.
If this project uses Poetry, uncomment the lines below:
  !pip install poetry
  !poetry install


In [7]:
# =========================
# Python path setup
# - supports repo-root imports
# - supports src/ layout
# =========================
import sys, pathlib

root = pathlib.Path(".").resolve()
src = root / "src"

# Add /src first if present
if src.exists():
    if str(src) not in sys.path:
        sys.path.insert(0, str(src))
    print("Added to PYTHONPATH:", src)

# Always add repo root as well
if str(root) not in sys.path:
    sys.path.insert(0, str(root))
print("Added to PYTHONPATH:", root)

# Quick sanity check
print("\nTop of sys.path:\n", "\n ".join(sys.path[:5]))


Added to PYTHONPATH: /content/REPO/src
Added to PYTHONPATH: /content/REPO

Top of sys.path:
 /content/REPO
 /content/REPO/src
 /content
 /env/python
 /usr/lib/python312.zip


## Run the project

Edit the next cell to match your repository entrypoint.

Examples:
- `!python run_analysis.py`
- `!python main.py --country Switzerland`
- `from package import run_full; run_full()`


In [8]:
# =========================
# PROJECT ENTRYPOINT
# =========================
# Example:
# !python run_analysis.py

print("Edit this cell to run your repository's entrypoint.")


Edit this cell to run your repository's entrypoint.
